In [8]:
import fasttext as ft
from gensim.models import FastText, Word2Vec
import MeCab as mc
import re
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.layers import Dense, Flatten, Dropout, LSTM, Embedding, Masking, GRU
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint

In [9]:
#load file Data_Intent.xlsx
def data(path, is_tsv):
    if is_tsv == True:
        data = pd.read_table(path)
    else:
        data = pd.read_excel(path)
    return data

data = data('Data_Intent.xlsx', False)
data.head()

#split data into 2 part: test part with the first row of each intent and train part in other.
data_train = data_test = pd.DataFrame(columns=data.columns)
for i in data['intent'].unique():
    train_ = data[data['intent']==i].iloc[1:, :]
    test_ = data[data['intent']==i].iloc[:1, :]
    data_train = pd.concat([data_train, train_], axis=0)
    data_test = pd.concat([data_test, test_], axis=0)
    data_train = data_train.reset_index(drop=True)
    data_test = data_test.reset_index(drop=True)

#Data0 is the merge of train parts and test parts
data0 = pd.concat([data_train, data_test], axis=0)
data0 = data0.reset_index(drop=True)
data0.tail()

,intent,sample question,answer,link to dialog
287,ic_stockPrice_date,日付で株価を知りたい,承知しました。指定の日時の、指定の銘柄の株価を調べます。,d_stockPrice_3_date
288,qa_shareIncShipDate,株主優待が届かない,優待が届く時期は、企業によって異なります。割当基準日の約3カ月後が多いようです。,NaN
289,qa_shareIncReceiveAvailability,どの会社も株主優待をもらえるか,全ての上場企業が株主優待を行っているわけではありません。,NaN
290,qa_shareIncReceiveConditions,株主優待をもらうための条件はなんですか。,以下の二つが条件となります。①権利付き最終日までに株主になっている事、②既定数以上の株式を保...,NaN
291,qa_ex-dividendDate,権利付き最終日とは,①権利付き最終日とは、株主としての権利を得るために株式を保有している必要がある期日で、この期...,NaN


In [10]:
#stop word in japanese: load into Japanese.txt
def stop_word(path):
    with open(path, 'r', encoding='utf-8') as file:
        content = file.read()
    stopword = content.split('\n')
    return stopword   

stopword = stop_word('./Japanese.txt')
print(stopword)

# split sentences into word and remove punc, stopword
def split_text(str_):
    str_ = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）：；《）《》“”()»〔〕-]+", "",str_)
    tag = mc.Tagger('-Ochasen')
    kekka = tag.parse(str_)
    lines = kekka.split('\n')
    word = []
    for line in lines:
        col = line.split('\t')
        if col[0] != None and col[0] != 'EOS' and col[0]!='':
            if col[0] not in stopword:
                word.append(col[0])
    return word


['\ufeffあそこ', 'あっ', 'あの', 'あのかた', 'あの人', 'あり', 'あります', 'ある', 'あれ', 'い', 'いう', 'います', 'いる', 'う', 'うち', 'え', 'お', 'および', 'おり', 'おります', 'か', 'かつて', 'から', 'が', 'き', 'ここ', 'こちら', 'こと', 'この', 'これ', 'これら', 'さ', 'さらに', 'し', 'しかし', 'する', 'ず', 'せ', 'せる', 'そこ', 'そして', 'その', 'その他', 'その後', 'それ', 'それぞれ', 'それで', 'た', 'ただし', 'たち', 'ため', 'たり', 'だ', 'だっ', 'だれ', 'つ', 'て', 'で', 'でき', 'できる', 'です', 'では', 'でも', 'と', 'という', 'といった', 'とき', 'ところ', 'として', 'とともに', 'とも', 'と共に', 'どこ', 'どの', 'な', 'なお', 'なかっ', 'ながら', 'なく', 'なっ', 'など', 'なに', 'なら', 'なり', 'なる', 'なん', 'に', 'において', 'における', 'について', 'にて', 'によって', 'により', 'による', 'に対して', 'に対する', 'に関する', 'の', 'ので', 'のみ', 'は', 'ば', 'へ', 'ほか', 'ほとんど', 'ほど', 'ます', 'また', 'または', 'まで', 'も', 'もの', 'ものの', 'や', 'よう', 'より', 'ら', 'られ', 'られる', 'れ', 'れる', 'を', 'ん', '何', '及び', '彼', '彼女', '我々', '特に', '私', '私達', '貴方', '貴方方']


In [11]:
#transfer sample question into list of word for train, test data
def transform_text(data):
    question_transform = data.apply(lambda x: split_text(x))
    return question_transform.values

question_transform = transform_text(data0['sample question'])
question_transform_train = transform_text(data_train['sample question'])
question_transform_test = transform_text(data_test['sample question'])

print(question_transform_test)

[list(['株主', '優待', '内容', '知り', 'たい'])
 list(['株主', '優待', '投資', '金額', '検索', 'たい'])
 list(['権利', '付き', '最終', '日', '株主', '優待', '絞り込み', 'たい'])
 list(['優待', '品', 'カテゴリー', '株主', '優待', '検索', 'たい'])
 list(['株価', '教え', 'ください'])
 list(['配当', '率', '高い', '銘柄', '株価', '検索', 'たい'])
 list(['日付', '株価', '知り', 'たい']) list(['株主', '優待', '届か', 'ない'])
 list(['会社', '株主', '優待', 'もらえる']) list(['株主', '優待', 'もらう', '条件'])
 list(['権利', '付き', '最終', '日'])]


In [12]:
#list of vocab in a columns dataframe

def listword(df):
    list_word = [i for j in df for i in j]
    return list_word
list_word = listword(question_transform)

#list vocab from sample question
def vocab(df):
    list_word_set = list(set(df))
    return list_word_set

list_word_set = vocab(list_word)
print(list_word_set)

['推移', '日', '期限', '引き', '日付', 'くれ', 'ください', '品', '検索', '権利', 'カテゴリー', '逆', '届く', '届か', '保有', '照会', '金額', 'くれる', '下回る', '内容', '付き', '牛', '受け', '株価', '配当', '抽出', '利回り', '届き', '落ち', 'ない', '特定', '月', '必要', 'もらえる', '遅い', '受け取る', 'どう', '優待', '絞り込み', '銘柄', 'たい', '株主', '資金', '高い', 'たら', '確定', '無い', '購入', '指定', 'もらえ', '高', '様', '予算', '会社', '込ん', 'よい', '最低', 'いつ', '情報', '企業', '条件', 'もらう', '定義', '率', '過去', '欲しい', 'ごろ', '最終', 'ジャンル', '調べ', '株式', '教え', '受ける', '株', '投資', '意味', '知り', 'やっ', '絞り']


In [13]:
#Creat word2vect model from gensim.Word2Vect
def word2vect():
    model = Word2Vec(question_transform_train)
    #check if folder 'model' if exist or not, if not exist then creat a folder 'model'
    if not os.path.exists('model'):
        os.makedirs('model')
    model.save('./model/word2vect.bin')
    return model

#Creat fasttext model from gensim.FastText
def fasttext():
    model = FastText(question_transform_train)
    #check if folder 'model' if exist or not, if not exist then creat a folder 'model'
    if not os.path.exists('model'):
        os.makedirs('model')
    model.save('./model/fasttext.bin')
    return model
word2vect()
fasttext()

In [14]:
#load model saved in above
def load_model0(model_name, path_model):
    model = model_name.load(path_model)
    return model

fasttext = load_model0(FastText, './model/fasttext.bin')
word2vect = load_model0(Word2Vec, './model/word2vect.bin')
vocab = fasttext.wv.vocab #list vocab in gensim --> not enough word from sample question list

not_enough = [i for i in list_word_set if i not in vocab]
print('list vocab in gensim:', list(vocab))
print('-'*50)
print('list vocab in sample question:', len(list_word_set))
print('-'*50)
print('The word is miss:', not_enough)

list vocab in gensim: ['株主', '優待', '内容', '教え', 'ください', '調べ', 'たい', '情報', '知り', '品', '株式', 'もらえ', 'もらえる', '企業', '投資', '金額', '絞り込み', '資金', '検索', '予算', '指定', '購入', '銘柄', '受け', '受ける', '権利', '付き', '最終', '日', 'もらう', '期限', '確定', '月', 'カテゴリー', '株価', '特定', '配当', '率', '高い', '絞り', '込ん', '高', '利回り', '日付', '過去', 'いつ', '条件']
--------------------------------------------------
list vocab in sample question: 79
--------------------------------------------------
The word is miss: ['推移', '引き', 'くれ', '逆', '届く', '届か', '保有', '照会', 'くれる', '下回る', '牛', '抽出', '届き', '落ち', 'ない', '必要', '遅い', '受け取る', 'どう', 'たら', '無い', '様', '会社', 'よい', '最低', '定義', '欲しい', 'ごろ', 'ジャンル', '株', '意味', 'やっ']


'届か' missing then a sentences is incorrect test

In [15]:
#Transfer label from column intent into one hot encoding
def one_hot_encodeing(feature):

    onehot = np.zeros((len(feature), feature.nunique()))
    feature_list = list(feature.unique())
    for intent in range(len(feature)):
        index = feature_list.index(feature[intent])
        onehot[intent, index] = 1
    return onehot, feature_list

train_onehot, train_feature = one_hot_encodeing(data_train['intent'])
test_onehot, test_feature = one_hot_encodeing(data_test['intent'])
onehot_, feature_ = one_hot_encodeing(data0['intent'])

In [16]:
#Creat a matrix from sample question with the dimention is row x max_lenght x 100
def word_embed(df, model, max_length):
    embed_all = []
    for token in df:
        embed_token=[]
        for word in token:
            if word in list(model.wv.vocab):
                word_vect = model.wv[word]
                embed_token.append(word_vect)
        
        if len(embed_token) > max_length: #if len(embed_token) > max_length then remove in larger
            embed_token = embed_token[:max_length]
        else: #if len(embed_token) < max_length then add 0 for enough
            add_vect = list(np.zeros((max_length - len(embed_token), 100)))
            embed_token = np.concatenate((embed_token, add_vect))
        embed_all.append(embed_token)
    embed_all = np.array(embed_all)
    return embed_all

all_ = word_embed(question_transform, fasttext, 10)
train = word_embed(question_transform_train, fasttext, 10)
test = word_embed(question_transform_test, fasttext, 10)
print(train.shape)
print(test.shape)

(281, 10, 100)
(11, 10, 100)


In [17]:
#creat LSTM model with input_shape = max_lengh x 100
def LSTM_(X, y, input_dim, epoch, batch_size):
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, input_shape=(input_dim)))
    model.add(Dropout(0.2))
    model.add(GRU(units=64, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32))
    model.add(Dropout(0.2))
    model.add(Dense(units=11, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='Adam', 
                 metrics=['accuracy'])
    if not os.path.exists('keras_model'):
        os.makedirs('keras_model')
    save_weight = os.path.join('keras_model', 'LSTM_weight.{acc:.4f}.hdf5')
    save_best = ModelCheckpoint(save_weight, monitor='acc', save_best_only=True, save_weights_only=False, mode='max', period=1)
    
    model.summary()
    
    history = model.fit(X, y, epochs=epoch, batch_size=batch_size, callbacks=[save_best])
    
    return model



Creat Model check points for model save best loss in each epoch, if loss new smaller than all file saved, it will save a new file

In [18]:
# train with epochs = 1000 and batch_size = 32
input_dim = (train.shape[1],train.shape[2])
model = LSTM_(train, train_onehot, input_dim, 1000, 32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 128)           117248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 10, 64)            37056     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 11)                363       
Total para

281/281 [==============================] - 0s 1ms/step - loss: 0.5322 - acc: 0.8043
Epoch 72/1000
281/281 [==============================] - 0s 957us/step - loss: 0.5427 - acc: 0.7900
Epoch 73/1000
281/281 [==============================] - 0s 824us/step - loss: 0.4991 - acc: 0.8327
Epoch 74/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.5003 - acc: 0.8434
Epoch 75/1000
281/281 [==============================] - 0s 873us/step - loss: 0.4791 - acc: 0.8292
Epoch 76/1000
281/281 [==============================] - 0s 989us/step - loss: 0.5166 - acc: 0.7829
Epoch 77/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.5268 - acc: 0.8149
Epoch 78/1000
281/281 [==============================] - 0s 950us/step - loss: 0.4919 - acc: 0.8185
Epoch 79/1000
281/281 [==============================] - 0s 2ms/step - loss: 0.5108 - acc: 0.8185
Epoch 80/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.5232 - acc: 0.8078
Epoch 81/1000
281/281 [=

281/281 [==============================] - 0s 961us/step - loss: 0.1672 - acc: 0.9466
Epoch 153/1000
281/281 [==============================] - 0s 875us/step - loss: 0.2255 - acc: 0.9288
Epoch 154/1000
281/281 [==============================] - 0s 894us/step - loss: 0.2608 - acc: 0.9110
Epoch 155/1000
281/281 [==============================] - 0s 875us/step - loss: 0.2224 - acc: 0.9075
Epoch 156/1000
281/281 [==============================] - 0s 774us/step - loss: 0.2120 - acc: 0.9324
Epoch 157/1000
281/281 [==============================] - 0s 914us/step - loss: 0.2087 - acc: 0.9217
Epoch 158/1000
281/281 [==============================] - 0s 838us/step - loss: 0.2059 - acc: 0.9359
Epoch 159/1000
281/281 [==============================] - 0s 848us/step - loss: 0.2693 - acc: 0.9075
Epoch 160/1000
281/281 [==============================] - 0s 814us/step - loss: 0.3015 - acc: 0.8968
Epoch 161/1000
281/281 [==============================] - 0s 890us/step - loss: 0.3010 - acc: 0.8826
Epoch

281/281 [==============================] - 0s 898us/step - loss: 0.1169 - acc: 0.9644
Epoch 234/1000
281/281 [==============================] - 0s 893us/step - loss: 0.1066 - acc: 0.9715
Epoch 235/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.1198 - acc: 0.9644
Epoch 236/1000
281/281 [==============================] - 0s 885us/step - loss: 0.1320 - acc: 0.9715
Epoch 237/1000
281/281 [==============================] - 0s 987us/step - loss: 0.1333 - acc: 0.9609
Epoch 238/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.1635 - acc: 0.9359
Epoch 239/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.1710 - acc: 0.9431
Epoch 240/1000
281/281 [==============================] - 0s 989us/step - loss: 0.1279 - acc: 0.9537
Epoch 241/1000
281/281 [==============================] - 0s 953us/step - loss: 0.1369 - acc: 0.9680
Epoch 242/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.1364 - acc: 0.9537
Epoch 243/100

281/281 [==============================] - 0s 1ms/step - loss: 0.0709 - acc: 0.9822
Epoch 316/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0837 - acc: 0.9751
Epoch 317/1000
281/281 [==============================] - 0s 903us/step - loss: 0.0802 - acc: 0.9680
Epoch 318/1000
281/281 [==============================] - 0s 883us/step - loss: 0.0821 - acc: 0.9751
Epoch 319/1000
281/281 [==============================] - 0s 825us/step - loss: 0.0867 - acc: 0.9751
Epoch 320/1000
281/281 [==============================] - 0s 868us/step - loss: 0.0709 - acc: 0.9751
Epoch 321/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0804 - acc: 0.9680
Epoch 322/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0786 - acc: 0.9680
Epoch 323/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0686 - acc: 0.9715
Epoch 324/1000
281/281 [==============================] - 0s 868us/step - loss: 0.0891 - acc: 0.9609
Epoch 325/1000


...

281/281 [==============================] - 0s 1ms/step - loss: 0.0309 - acc: 0.9893
Epoch 639/1000
281/281 [==============================] - 0s 860us/step - loss: 0.0406 - acc: 0.9858
Epoch 640/1000
281/281 [==============================] - 0s 817us/step - loss: 0.0366 - acc: 0.9858
Epoch 641/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0354 - acc: 0.9822
Epoch 642/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0454 - acc: 0.9751
Epoch 643/1000
281/281 [==============================] - 0s 885us/step - loss: 0.0252 - acc: 0.9893
Epoch 644/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0302 - acc: 0.9929
Epoch 645/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0323 - acc: 0.9858
Epoch 646/1000
281/281 [==============================] - 0s 889us/step - loss: 0.0484 - acc: 0.9786
Epoch 647/1000
281/281 [==============================] - 0s 888us/step - loss: 0.0365 - acc: 0.9822
Epoch 648/1000


281/281 [==============================] - 0s 900us/step - loss: 0.0320 - acc: 0.9893
Epoch 720/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0337 - acc: 0.9893
Epoch 721/1000
281/281 [==============================] - 0s 878us/step - loss: 0.0334 - acc: 0.9858
Epoch 722/1000
281/281 [==============================] - 0s 956us/step - loss: 0.0289 - acc: 0.9786
Epoch 723/1000
281/281 [==============================] - 0s 890us/step - loss: 0.0377 - acc: 0.9893
Epoch 724/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0341 - acc: 0.9858
Epoch 725/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0347 - acc: 0.9786
Epoch 726/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0346 - acc: 0.9893
Epoch 727/1000
281/281 [==============================] - 1s 2ms/step - loss: 0.0318 - acc: 0.9822
Epoch 728/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0286 - acc: 0.9893
Epoch 729/1000
28

281/281 [==============================] - 0s 1ms/step - loss: 0.0359 - acc: 0.9822
Epoch 801/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0426 - acc: 0.9751
Epoch 802/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0295 - acc: 0.9929
Epoch 803/1000
281/281 [==============================] - 0s 947us/step - loss: 0.0479 - acc: 0.9786
Epoch 804/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0436 - acc: 0.9822
Epoch 805/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0366 - acc: 0.9858
Epoch 806/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0359 - acc: 0.9786
Epoch 807/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0462 - acc: 0.9822
Epoch 808/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0462 - acc: 0.9822
Epoch 809/1000
281/281 [==============================] - 0s 936us/step - loss: 0.0399 - acc: 0.9786
Epoch 810/1000
281/28

281/281 [==============================] - 0s 925us/step - loss: 0.0843 - acc: 0.9644
Epoch 882/1000
281/281 [==============================] - 0s 829us/step - loss: 0.0795 - acc: 0.9537
Epoch 883/1000
281/281 [==============================] - 0s 838us/step - loss: 0.0767 - acc: 0.9680
Epoch 884/1000
281/281 [==============================] - 0s 993us/step - loss: 0.1181 - acc: 0.9537
Epoch 885/1000
281/281 [==============================] - 0s 972us/step - loss: 0.1540 - acc: 0.9359
Epoch 886/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.2911 - acc: 0.9004
Epoch 887/1000
281/281 [==============================] - 0s 890us/step - loss: 0.3020 - acc: 0.9039
Epoch 888/1000
281/281 [==============================] - 0s 834us/step - loss: 0.2181 - acc: 0.9253
Epoch 889/1000
281/281 [==============================] - 0s 832us/step - loss: 0.1836 - acc: 0.9431
Epoch 890/1000
281/281 [==============================] - 0s 901us/step - loss: 0.0642 - acc: 0.9751
Epoch 8

281/281 [==============================] - 0s 911us/step - loss: 0.0375 - acc: 0.9715
Epoch 963/1000
281/281 [==============================] - 0s 829us/step - loss: 0.0331 - acc: 0.9858
Epoch 964/1000
281/281 [==============================] - 0s 878us/step - loss: 0.0306 - acc: 0.9858
Epoch 965/1000
281/281 [==============================] - 0s 888us/step - loss: 0.0511 - acc: 0.9786
Epoch 966/1000
281/281 [==============================] - 0s 975us/step - loss: 0.0270 - acc: 0.9822
Epoch 967/1000
281/281 [==============================] - 0s 930us/step - loss: 0.0320 - acc: 0.9822
Epoch 968/1000
281/281 [==============================] - 0s 972us/step - loss: 0.0232 - acc: 0.9893
Epoch 969/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0227 - acc: 0.9893
Epoch 970/1000
281/281 [==============================] - 0s 1ms/step - loss: 0.0252 - acc: 0.9858
Epoch 971/1000
281/281 [==============================] - 0s 993us/step - loss: 0.0313 - acc: 0.9751
Epoch 972

In [19]:
#sort the model saved and load the best model is the first list file
def load_(path):
    list_ = os.listdir(path)
    list_.sort()
    file_name = os.path.join(path, list_[-1])
    print(file_name)
    model_predict = load_model(file_name, '')
    return model_predict

model_predict = load_('keras_model')

keras_model\LSTM_weight.1.0000.hdf5


In [20]:
#evaluate model
def evaluate_model(X, y, model):
    loss = model.evaluate(X, y, verbose=0)
    return loss

evaluate_model(test, test_onehot, model_predict)

[0.34463661909103394, 0.9090909361839294]

In [21]:
def predict(sent, model, model_predict, max_length, threshold):
    if sent == '':
        result = '523'
    else:
        split_ = split_text(sent)
        vect = word_embed([split_], model, max_length)
        predict = model_predict.predict(vect)
        max_prob = np.max(predict)
        if max_prob >= threshold: #add threshold to predict sent
            result = train_feature[np.argmax(predict[0])]
        else:
            result = '216542'
    return result

display again result of test data

In [23]:
for index in range(data_test.shape[0]):
    sent = data_test.loc[index, 'sample question']
    actual = data_test.loc[index, 'intent']
    pre = predict(sent, fasttext, model_predict, 10, 0.5)
    print('sentences:', sent)
    print('predict:', pre)
    print('actual:', actual)


sentences: 株主優待の内容を知りたいです
predict: ic_incentives_stockName
actual: ic_incentives_stockName
sentences: 株主優待を投資金額から検索したい
predict: ic_incentives_investmentAmount
actual: ic_incentives_investmentAmount
sentences: 権利付き最終日から株主優待を絞り込みたい
predict: ic_incentives_month
actual: ic_incentives_month
sentences: 優待品のカテゴリーから株主優待を検索したい
predict: ic_incentives_category
actual: ic_incentives_category
sentences: 株価を教えてください
predict: ic_stockPrice_stockName
actual: ic_stockPrice_stockName
sentences: 配当率の高い銘柄の株価を検索したい
predict: ic_stockPrice_dividendRate
actual: ic_stockPrice_dividendRate
sentences: 日付で株価を知りたい
predict: ic_stockPrice_date
actual: ic_stockPrice_date
sentences: 株主優待が届かない
predict: qa_shareIncReceiveConditions
actual: qa_shareIncShipDate
sentences: どの会社も株主優待をもらえるか
predict: qa_shareIncReceiveAvailability
actual: qa_shareIncReceiveAvailability
sentences: 株主優待をもらうための条件はなんですか。
predict: qa_shareIncReceiveConditions
actual: qa_shareIncReceiveConditions
sentences: 権利付き最終日とは
predict: qa_ex-dividendDate
actu